In [ ]:
data_and_output_dir='/Users/ansonkong/Downloads/Data for nyu work/'
full_future_data_path='/Users/ansonkong/Downloads/rcp85hotter/'
full_historical_data_path='/Users/ansonkong/Downloads/historic/'


mapping_path = data_and_output_dir + 'output/merged_rb_control_area_mapping.csv'
future_population_data_path = data_and_output_dir + 'input/Electric_Retail_Service_Territories/ssp3_county_population.csv'
historical_population_data_path=data_and_output_dir + 'input/county_populations_2000_to_2020.csv'




In [ ]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import geopandas as gpd
from datetime import datetime
# Load control area mapping
control_area_path = '/Users/ansonkong/Downloads/county_control_area_mapping_no_state_fips.csv'
control_area_df = pd.read_csv(control_area_path)

# Load meteorology data
meteorology_path = '/Users/ansonkong/Downloads/rcp85hotter/2060/2060_01_01_02_UTC_County_Mean_Meteorology.csv'
meteorology_df = pd.read_csv(meteorology_path)
# Calculate wind speed
meteorology_df['WSPD'] = np.sqrt(meteorology_df['U10']**2 + meteorology_df['V10']**2)
# Add 'year' and 'UTC time' to meteorology data
file_name = os.path.basename(meteorology_path)  # Gets '2060_01_01_01_UTC_County_Mean_Meteorology.csv'
year_utc_time = file_name.split('_UTC')[0]  # Gets '2060_01_01_01'
year = year_utc_time.split('_')[0]  # Gets '2060'
utc_time = '_'.join(year_utc_time.split('_')[:4])  # Gets '2060_01_01_01'
# Parse the original string to a datetime object
dt = datetime.strptime(utc_time, '%Y_%m_%d_%H')

# Format the datetime object to the desired format
utc_time= dt.strftime('%-m/%-d/%Y %-I:%M:%S %p')
meteorology_df['year'] = year
meteorology_df['Time_UTC'] = utc_time
# Merge on FIPS
merged_df = pd.merge(control_area_df, meteorology_df, on='FIPS', how='inner')
# Assuming population_data is loaded and prepared similarly
# Perform interpolation (example shown for clarity; adjust as needed)
years = np.arange(2020, 2101, 10)
interp_years = np.arange(2020, 2101)

for index, row in merged_df.iterrows():
    pop_values = row[years.astype(str)].values
    interpolator = interp1d(years, pop_values, kind='linear', fill_value="extrapolate")
    interpolated_values = interpolator(interp_years)
    population_data.loc[index, interp_years.astype(str)] = interpolated_values
# Example of grouping and applying weighted average
def weighted_average(group, columns):
    year_col = str(group['year'].iloc[0])  # Assuming year is constant within each group
    weights = group[year_col]  # This assumes a direct match, which might need adjustment
    return {col: np.average(group[col], weights=weights) for col in columns}

# Assuming 'columns' is a list of meteorology columns to aggregate
columns = meteorology_df.columns.difference(['FIPS', 'year', 'Time_UTC'])

# Apply the weighted average within each group
grouped = merged_df.groupby([ 'Majority_CNTRL_AREA', 'year', 'Time_UTC']).apply(lambda x: weighted_average(x, columns)).reset_index()


expanded_grouped = grouped[0].apply(pd.Series)

grouped_expanded = grouped.drop(columns=[0]).join(expanded_grouped)

# 'grouped_expanded' now contains the original grouping columns and the expanded meteorology columns
grouped_expanded

In [2]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import geopandas as gpd
from datetime import datetime
import os
from glob import glob
debug=False
# Define paths
data_and_output_dir = '/Users/ansonkong/Downloads/Data for nyu work/'
mapping_path = os.path.join(data_and_output_dir, 'output/merged_rb_control_area_mapping.csv')
future_population_data_path = os.path.join(data_and_output_dir, 'input/Electric_Retail_Service_Territories/ssp3_county_population.csv')
historical_population_data_path = os.path.join(data_and_output_dir, 'input/county_populations_2000_to_2020.csv')

# Load mapping and population data
mapping_df = pd.read_csv(mapping_path)
future_population_df = pd.read_csv(future_population_data_path)
historical_population_df = pd.read_csv(historical_population_data_path)


# Function to process a single meteorology file
def process_meteorology_file(file_path, population_df, mapping_df, is_future=True):
    # Load meteorology data
    meteorology_df = pd.read_csv(file_path)
    
    # Calculate wind speed
    meteorology_df['WSPD'] = np.sqrt(meteorology_df['U10']**2 + meteorology_df['V10']**2)
    
    # Extract date and time from file name
    file_name = os.path.basename(file_path)
    year, month, day, hour = map(int, file_name.split('_')[:4])
    
    # Format UTC time
    utc_time = datetime(year, month, day, hour).strftime('%Y-%m-%d %H:%M:%S')
    meteorology_df['Time_UTC'] = utc_time
    
    # Merge with mapping
    merged_df = pd.merge(mapping_df, meteorology_df, left_on='GEOID', right_on='FIPS', how='inner')

    # Apply population weighting if future data
    if is_future:
        # Assuming future_population_df has columns for years and population projections
        # Interpolate population data for the specific year
        population_df['Year'] = population_df['Year'].astype(int)
        pop_interpolator = interp1d(population_df['Year'], population_df[str(year)], kind='linear', fill_value="extrapolate")
        merged_df['Population'] = pop_interpolator(year)
    else:
        # For historical data, directly use the population for the year
        merged_df = pd.merge(merged_df, population_df[['FIPS', f'pop_{year}']], on='FIPS', how='left')
        merged_df.rename(columns={f'pop_{year}': 'Population'}, inplace=True)

    # Ensure population column exists and handle any NaNs
    merged_df['Population'].fillna(0, inplace=True)

    # Group by RB and compute weighted averages for meteorological variables
    def weighted_avg(group, var):
        return np.average(group[var], weights=group['Population'])

    variables = ['T2', 'Q2', 'SWDOWN', 'GLW', 'WSPD']
    grouped = merged_df.groupby('rb').apply(lambda x: pd.Series({var: weighted_avg(x, var) for var in variables}))
    grouped['Time_UTC'] = utc_time
    grouped.reset_index(inplace=True)
    
    # Output file path
    output_file = f"{data_and_output_dir}output/{'future' if is_future else 'historical'}/{grouped['rb'][0]}_WRF_Hourly_Mean_Meteorology_{year}.csv"
    grouped.to_csv(output_file, index=False)
    print(f"Processed and saved: {output_file}")
# Example usage for a single file
process_meteorology_file(
    file_path=os.path.join(data_and_output_dir, 'rcp85hotter/2060/2060_01_01_02_UTC_County_Mean_Meteorology.csv'),
    population_df=future_population_df,
    mapping_df=mapping_df,
    is_future=True
)

# Loop through all files in the future and historical directories
for file_path in glob(os.path.join(data_and_output_dir, 'rcp85hotter/*/*.csv')):
    process_meteorology_file(file_path, future_population_df, mapping_df, is_future=True)

for file_path in glob(os.path.join(data_and_output_dir, 'historic/*/*.csv')):
    process_meteorology_file(file_path, historical_population_df, mapping_df, is_future=False)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/ansonkong/Downloads/Data for nyu work/rcp85hotter/2060/2060_01_01_02_UTC_County_Mean_Meteorology.csv'

In [ ]:

def process_meteorology_file(file_path, population_df, mapping_df, is_future):
    # Load meteorology data
    meteorology_df = pd.read_csv(file_path)
    meteorology_df['WSPD'] = np.sqrt(meteorology_df['U10']**2 + meteorology_df['V10']**2)
    
    # Extract year, month, day, hour from file name
    print(os.path.basename(file_path).split('_'))
    year, month, day, hour, _, _, _, _ = os.path.basename(file_path).split('_')
    
    utc_time = datetime(int(year), int(month), int(day), int(hour[:-3])).strftime('%Y-%m-%d %H:%M:%S')
    meteorology_df['Time_UTC'] = utc_time
    
    # Merge meteorology data with mapping
    merged_df = pd.merge(meteorology_df, mapping_df, left_on='FIPS', right_on='GEOID', how='inner')
    
    # Apply population weighting
    if is_future:
        # Interpolate future population data
        year_col = f'pop_{year}'
        merged_df = merged_df.merge(future_population_df[['GEOID', year_col]], on='GEOID')
        merged_df['Population'] = merged_df[year_col]
    else:
        # Use historical population data
        merged_df = merged_df.merge(historical_population_df[['GEOID', f'pop_{year}']], on='GEOID')
        merged_df['Population'] = merged_df[f'pop_{year}']
    
    # Compute weighted averages
    weighted_avg_cols = ['T2', 'Q2', 'SWDOWN', 'GLW', 'WSPD']
    for col in weighted_avg_cols:
        merged_df[col] = merged_df[col] * merged_df['Population']
    grouped = merged_df.groupby('rb')[weighted_avg_cols + ['Population']].sum()
    for col in weighted_avg_cols:
        grouped[col] = grouped[col] / grouped['Population']
    
    # Save processed data by RB and year
    for rb, group in grouped.iterrows():
        output_file = f"{data_and_output_dir}output/{'future' if is_future else 'historical'}/{rb}_WRF_Hourly_Mean_Meteorology_{year}.csv"
        group.drop('Population').to_csv(output_file, header=True)
        print(f"Processed and saved: {output_file}")
def process_all_files():
    for file_path in glob(f"{full_future_data_path}/**/*.csv", recursive=True):
        process_meteorology_file(file_path, future_population_df, mapping_df, is_future=True)
    for file_path in glob(f"{full_historical_data_path}/**/*.csv", recursive=True):
        process_meteorology_file(file_path, historical_population_df, mapping_df, is_future=False)

process_all_files()


In [3]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import os
from datetime import datetime
from glob import glob
debug=False
# Define paths
data_and_output_dir = '/Users/ansonkong/Downloads/Data for nyu work/'
full_future_data_path = '/Users/ansonkong/Downloads/rcp85hotter/'
full_historical_data_path = '/Users/ansonkong/Downloads/historic/'

#this just smooth out operation
cutoff_year=2020 #The year prediction dataset start
start_year=2000 #The year historical 
end_year=2100



#Path no need to update
mapping_path = data_and_output_dir + 'output/merged_rb_control_area_mapping.csv'
future_population_data_path = data_and_output_dir + 'input/Electric_Retail_Service_Territories/ssp3_county_population.csv'
historical_population_data_path = data_and_output_dir + 'input/county_populations_2000_to_2020.csv'
output_path = data_and_output_dir + 'output/'

# Load mapping and population data
mapping_df = pd.read_csv(mapping_path)
future_population_df = pd.read_csv(future_population_data_path)
historical_population_df = pd.read_csv(historical_population_data_path)

years = np.arange(cutoff_year, end_year + 1, 10)
interp_years = np.arange(cutoff_year, end_year + 1)

# Ensure FIPS codes are zero-padded strings
historical_population_df['county_FIPS'] = historical_population_df['county_FIPS'].astype(str).str.zfill(5)
future_population_df['FIPS'] = future_population_df['FIPS'].astype(str).str.zfill(5)
mapping_df['GEOID'] = mapping_df['GEOID'].astype(str).str.zfill(5)

# Rename 'pop_{year}' columns to just '{year}'
for year in range(start_year, cutoff_year+1):
    if f'pop_{year}' in historical_population_df.columns:
        historical_population_df.rename(columns={f'pop_{year}': str(year)}, inplace=True)



# Assuming 'FIPS' is the column to join on and 'Year' is the column indicating the year in future_population_df
for index, row in future_population_df.iterrows():
    pop_values = [row[str(year)] for year in years if str(year) in row]
    interpolator = interp1d(years, pop_values, kind='linear', fill_value="extrapolate")
    interpolated_values = interpolator(interp_years)
    for year in interp_years:
        future_population_df.at[index, str(year)] = interpolated_values[year - cutoff_year]


# Select relevant columns for historical data
historical_population_df['FIPS']=historical_population_df['county_FIPS']
historical_population_df = historical_population_df[['FIPS'] + [str(year) for year in range(start_year, cutoff_year)]]

# Select relevant columns for future data
future_population_df = future_population_df[['FIPS'] + [str(year) for year in interp_years]]

# Concatenate historical and future dataframes
combined_population_df = pd.concat([historical_population_df, future_population_df]).drop_duplicates(subset=['FIPS']).reset_index(drop=True)
# Merge combined population data with mapping data
# Assuming 'GEOID' in mapping_df corresponds to 'FIPS' in population data
combined_df = pd.merge(mapping_df, combined_population_df, left_on='GEOID', right_on='FIPS', how='inner')
combined_df['FIPS']=combined_df['GEOID']
# Identify FIPS codes in mapping_df that did not successfully join
missing_in_combined = set(mapping_df['GEOID']) - set(combined_df['GEOID'])
print(missing_in_combined)
for column in combined_df.columns:
    print(column)

set()
STATEFP
COUNTYFP
COUNTYNS
AFFGEOID
GEOID
NAME
LSAD
ALAND
AWATER
geometry
FIPS_x
rb
is_mainland
CNTRL_AREA
FIPS_y
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
FIPS


In [5]:
# Ensure output directories exist
debug=True
historical_weather_path = os.path.join(output_path, 'historical_weather')
future_weather_path = os.path.join(output_path, 'future_weather')
os.makedirs(historical_weather_path, exist_ok=True)
os.makedirs(future_weather_path, exist_ok=True)
yearloopbreaker=False
fileloopbreaker=False
rbloopbreaker=False
for year in range(start_year, end_year + 1):

    if yearloopbreaker:break


    print(f'Process start for year: {year}')
    if year <= cutoff_year:
        folder_to_read = os.path.join(full_historical_data_path, str(year))
        folder_to_store = historical_weather_path
    else:
        folder_to_read = os.path.join(full_future_data_path, str(year))
        folder_to_store = future_weather_path
    
    # Initialize a dictionary to hold DataFrames for each rb
    rb_dfs = {}

    # Iterate through each file in the year's folder
    for file_path in glob(os.path.join(folder_to_read, '*.csv')):
        
        if fileloopbreaker:break
        
        meteorology_df = pd.read_csv(file_path)
        #print(meteorology_df.columns)
        meteorology_df['FIPS'] = meteorology_df['FIPS'].astype(str).str.zfill(5)
        meteorology_df['WSPD'] = np.sqrt(meteorology_df['U10']**2 + meteorology_df['V10']**2)
        
        # Extract date and time from file name
        _, month, day, hour = map(int, os.path.basename(file_path).split('_')[:4])
        utc_time = datetime(year, month, day, hour).strftime('%Y-%m-%d %H:%M:%S')
        meteorology_df['Time_UTC'] = utc_time
        
        # Merge with background data
        merged_df = pd.merge(combined_df, meteorology_df, on='FIPS', how='inner')
        
        # Calculate weighted averages and accumulate in rb_dfs
        merged_df['Population'] = merged_df[str(year)]
        for rb, group in merged_df.groupby('rb'):
            if rbloopbreaker:break
            
            if rb not in rb_dfs:
                rb_dfs[rb] = pd.DataFrame()
            weighted = group.copy()
            for col in ['T2', 'Q2', 'SWDOWN', 'GLW', 'WSPD']:
                weighted[col] = weighted[col] * weighted['Population']
            weighted_sum = weighted.groupby('rb').sum()
            for col in ['T2', 'Q2', 'SWDOWN', 'GLW', 'WSPD']:
                weighted_sum[col] = weighted_sum[col] / weighted_sum['Population']
            weighted_sum['Time_UTC'] = utc_time
            rb_dfs[rb] = pd.concat([rb_dfs[rb], weighted_sum.reset_index()], ignore_index=True)
        if debug:
            fileloopbreaker=True
            yearloopbreaker=True

    
    # Save each RB's accumulated data to its file
    for rb, df in rb_dfs.items():
        output_file = os.path.join(folder_to_store, f"{rb}_WRF_Hourly_Mean_Meteorology_{year}.csv")
        df.to_csv(output_file, index=False)
        print(f"Processed and saved: {output_file} for RB: {rb}")

Process start for year: 2000
Processed and saved: /Users/ansonkong/Downloads/Data for nyu work/output/historical_weather/p1_WRF_Hourly_Mean_Meteorology_2000.csv for RB: p1
Processed and saved: /Users/ansonkong/Downloads/Data for nyu work/output/historical_weather/p10_WRF_Hourly_Mean_Meteorology_2000.csv for RB: p10
Processed and saved: /Users/ansonkong/Downloads/Data for nyu work/output/historical_weather/p100_WRF_Hourly_Mean_Meteorology_2000.csv for RB: p100
Processed and saved: /Users/ansonkong/Downloads/Data for nyu work/output/historical_weather/p101_WRF_Hourly_Mean_Meteorology_2000.csv for RB: p101
Processed and saved: /Users/ansonkong/Downloads/Data for nyu work/output/historical_weather/p102_WRF_Hourly_Mean_Meteorology_2000.csv for RB: p102
Processed and saved: /Users/ansonkong/Downloads/Data for nyu work/output/historical_weather/p103_WRF_Hourly_Mean_Meteorology_2000.csv for RB: p103
Processed and saved: /Users/ansonkong/Downloads/Data for nyu work/output/historical_weather/p10